In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.06627869606018066
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 8.332556517675886
200000 13.149748603618796
300000 12.04095063280132
400000 12.556124986306529
500000 13.623885820553125
600000 18.181107500897866
700000 18.09072369496429
800000 21.483696177564262
900000 22.979102287040202
1000000 21.708160720067752
1100000 18.674535405082352
1200000 10.03325148712185
1300000 21.677862375627914
1400000 21.286247477166402
1500000 21.60236490837913
1600000 21.601208626478165
1700000 22.796183114623943
1800000 22.32805379034922
1900000 21.355613818683686
2000000 22.206853457351905
2100000 22.25738165393538
2200000 21.96255928508365
2300000 5.845769448476775
2400000 24.439921331211504
2500000 28.213416823530434
2600000 27.304786551950937
2700000 27.713311213396324
2800000 27.521022592364286
2900000 27.367104968667487
3000000 2

29400000 21.52919673620499
29500000 7.571950955390544
29600000 22.445764431003646
29700000 20.74921425207913
29800000 21.927183258514834
29900000 20.809834232158376
30000000 21.322401587155973
30100000 20.627589152991604
30200000 19.5589688307708
30300000 20.18341940973874
30400000 5.8563689168686865
30500000 20.25501849137102
30600000 19.610830032938363
30700000 19.961598187576442
30800000 21.471286370738387
30900000 19.924569786755224
31000000 21.281182247427125
31100000 20.84799221027598
31200000 21.201251081684735
31300000 8.874613660186023
31400000 20.941850829549622
31500000 22.112489065767043
31600000 21.676453985794108
31700000 19.047185242329522
31800000 12.08928223708448
31900000 20.46312303787305
32000000 22.1639238794913
32100000 20.071235591143527
32200000 21.394208047332466
32300000 20.262947377145462
32400000 20.01942980831894
32500000 19.786032629825375
32600000 12.393290095227208
32700000 14.198539374672505
32800000 19.079522288843243
32900000 20.72660389067967
3300000

59200000 17.089932505586567
59300000 17.588213287521477
59400000 16.354691888278694
59500000 16.97604714842977
59600000 7.182276111053384
59700000 15.210772820814643
59800000 17.3404642400603
59900000 15.969768328602461
60000000 17.845671169949714
60100000 17.534636832945942
60200000 17.836380114239976
60300000 7.90561477503171
60400000 18.163382330067506
60500000 20.1724498482165
60600000 18.396254930413516
60700000 19.133280560588577
60800000 19.837903014717657
60900000 19.18733249877069
61000000 20.23524780669084
61100000 18.71493133728407
61200000 9.245784159502083
61300000 17.36015547530689
61400000 18.307493202855422
61500000 19.008805807728987
61600000 17.491900278239836
61700000 17.784334257534262
61800000 14.346526078938595
61900000 8.612568291528941
62000000 17.35337082171007
62100000 16.850457784900897
62200000 17.30573347362871
62300000 16.90884980952812
62400000 18.6858984504733
62500000 9.318068363076408
62600000 16.341717313983214
62700000 19.17491576953094
62800000 18.8

89000000 14.759529728972892
89100000 14.372366435489438
89200000 14.81824664919413
89300000 6.340403816983425
89400000 15.200494173196054
89500000 15.05677036491708
89600000 15.520542560924145
89700000 15.595368631453745
89800000 15.743799912828415
89900000 16.420124317832688
90000000 16.019296486460902
90100000 15.762660531382293
90200000 15.745903031925222
90300000 15.554501847608039
90400000 15.32775355509592
90500000 14.174405932413666
90600000 12.367433255853674
90700000 8.38565925453338
90800000 13.898902000255422
90900000 14.000493182061785
91000000 14.227682347101469
91100000 4.024305913040072
91200000 14.60944022627332
91300000 14.32270236157632
91400000 14.114356051320215
91500000 7.988823414495521
91600000 12.90655132114023
91700000 14.222668684819828
91800000 15.295736790928167
91900000 15.12285850755178
92000000 15.524915606601411
92100000 15.62750041593791
92200000 15.661921155724961
92300000 15.98817610751093
92400000 15.69922322236304
92500000 15.478835672678589
9260000

118000000 10.490214580374044
118100000 5.829494017523895
118200000 12.50201803676337
118300000 5.507486859129409
118400000 12.677943378872422
118500000 12.806395718286007
118600000 12.534348663927375
118700000 12.893036197562667
118800000 12.648815640249637
118900000 12.953550659478228
119000000 13.59451241114115
119100000 14.060856215248911
119200000 14.881667356403916
119300000 14.077614397251084
119400000 13.49651672329447
119500000 12.913635452210993
119600000 12.717288758361425
119700000 13.233155994396112
119800000 12.421458434113733
119900000 12.906632085263897
120000000 12.328107102361322
120100000 5.966181385300924
120200000 12.044475826626453
120300000 5.131183715115166
120400000 11.233756696406514
120500000 6.929328550763328
120600000 11.029927750517867
120700000 13.04524560548232
120800000 12.327385671543736
120900000 13.057122694315524
121000000 12.388079949946702
121100000 12.852750307061457
121200000 13.369356593735993
121300000 13.82931450163731
121400000 14.17934814358

...

175600000 9.769310879420429
175700000 8.925914006972352
175800000 8.767916084588746
175900000 8.879000411206492
176000000 8.91743771684826
176100000 8.906090062719265
176200000 8.628026118317486
176300000 3.7060049689049652
176400000 8.811996471048982
176500000 3.6890736301546894
176600000 7.222762539009525
176700000 6.651818279504309
176800000 6.0491803891506395
176900000 8.64086231360743
177000000 9.222693193835113
177100000 8.146079474169314
177200000 9.68874217222783
177300000 8.351681303497681
177400000 9.402043062635322
177500000 9.243984202884462
177600000 9.724486604565806
177700000 9.497857845147323
177800000 9.689711573844496
177900000 8.985238262800323
178000000 8.366386570901602
178100000 9.102200231220696
178200000 8.33697950086481
178300000 8.738825655203273
178400000 8.296954961738182
178500000 5.137491164258768
178600000 7.876430884225319
178700000 6.613359416641593
178800000 3.918482946585904
178900000 8.881569963484079
179000000 3.525416209028727
179100000 8.156533991

205000000 7.523924470932841
205100000 6.688687717316666
205200000 7.052333602012244
205300000 2.725351988321503
205400000 6.690817097797533
205500000 6.870959041715264
205600000 6.402565804674914
205700000 4.23706802330044
205800000 4.999849976241218
205900000 6.427884046284973
206000000 7.003843113735659
206100000 6.628750993743573
206200000 2.4047471065353356
206300000 7.128348443894303
206400000 6.658367043630199
206500000 7.4404675040589865
206600000 7.095684185263027
206700000 7.732683025156081
206800000 7.093246394902737
206900000 7.297060413473696
207000000 6.99882112558297
207100000 6.95482196401448
207200000 6.903534157097316
207300000 6.4311256705962085
207400000 6.866780323184778
207500000 3.8123130464938613
207600000 5.2059848293563995
207700000 6.596409509437501
207800000 6.010848923162004
207900000 6.839231487322538
208000000 1.7850512225670914
208100000 6.3225024874614375
208200000 6.471391313962797
208300000 6.122143531829715
208400000 5.119405097877572
208500000 4.0901

234200000 5.132919275635282
234300000 1.9180089986299904
234400000 4.9348932229509055
234500000 4.860874928946131
234600000 4.4753761029568215
234700000 4.686161339245498
234800000 4.528177930010415
234900000 3.996030443822692
235000000 2.1314264220400356
235100000 4.396334833905187
235200000 4.728334685615573
235300000 4.445803828933745
235400000 4.670710597496579
235500000 4.9588170053217375
235600000 3.237043326245725
235700000 3.77315620611863
235800000 5.0097432179313754
235900000 5.208993841804704
236000000 5.080856029287954
236100000 5.101341824882485
236200000 5.191803145884296
236300000 4.881050111409882
236400000 5.361618354842027
236500000 2.9334438947679105
236600000 4.000323340979076
236700000 4.793161562209507
236800000 4.511363706188908
236900000 4.692088611828397
237000000 4.341191083201925
237100000 4.52300671246177
237200000 1.8536575546974374
237300000 4.71485351039704
237400000 4.809986619759053
237500000 4.440165244561073
237600000 5.1171474326552975
237700000 4.84

263100000 2.6831738061350188
263200000 2.825582729293485
263300000 1.0872573004459771
263400000 2.8119445157900946
263500000 2.899452226928085
263600000 2.608430515697863
263700000 2.8967650165072647
263800000 2.5571761913335025
263900000 2.678279268858734
264000000 2.571418531336493
264100000 2.510131490983695
264200000 0.7210945741346744
264300000 2.7213725033942797
264400000 2.9077531304570137
264500000 2.7128829794047413
264600000 2.641333919118474
264700000 2.795784822719773
264800000 2.6275683523314597
264900000 2.8085305824141904
265000000 2.8440610139952005
265100000 0.9107111396367518
265200000 2.7681588038169673
265300000 2.546200035861105
265400000 2.6634387432434496
265500000 1.5554765532661112
265600000 1.9708746396649879
265700000 2.553920129472402
265800000 2.5410642371413874
265900000 2.4708739666304096
266000000 2.3721135162613356
266100000 2.504661135834664
266200000 2.29472188760829
266300000 2.2770452503182788
266400000 1.4584522635883193
266500000 1.565514576763093

291600000 0.3527351917263799
291700000 0.31588331198471786
291800000 0.3645389052529964
291900000 0.3125634070476691
292000000 0.3228540070266525
292100000 0.2836998620899783
292200000 0.30931500415368546
292300000 0.07063557983115315
292400000 0.26832967507993843
292500000 0.2184143375969827
292600000 0.20523793460962172
292700000 0.2145463510379427
292800000 0.1735538366603421
292900000 0.23500406760028997
293000000 0.21357677196929853
293100000 0.22239875951545104
293200000 0.17417930907556411
293300000 0.1686263387194044
293400000 0.06817227649419175
293500000 0.12514728776859244
293600000 0.15579683106640646
293700000 0.14831223335705862
293800000 0.1874137201416691
293900000 0.1556431919171446
294000000 0.1741408482326203
294100000 0.14791896623075007
294200000 0.16501901416822937
294300000 0.09505721303429868
294400000 0.13384211796787382
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.05058924149457283
294600000 0.007802078855852286
294700000 